# Section 6 — MIRCF Reproduction Notebook

This notebook reproduces the empirical results reported in **Section 6** of the MIRCF paper.

- Figure numbering follows the paper (Fig. 6.1+)
- All outputs are written to a controlled `outputs/` directory
- The notebook is CI-safe and artifact-review ready


In [ ]:
# --- GLOBAL SETUP (artifact-safe) ---
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

# Base directory = repository root (works in CI and locally)
BASE_DIR = Path.cwd()

# Canonical outputs directory
EXPORT_DIR = BASE_DIR / "outputs"
EXPORT_DIR.mkdir(parents=True, exist_ok=True)

FIG_DPI = 300

plt.rcParams.update({
    "figure.dpi": FIG_DPI,
    "font.size": 11
})

print(f"Outputs directory: {EXPORT_DIR.resolve()}")

## Figure 6.1 — Overview Across RQ1–RQ6

This figure summarizes the **final Lyapunov energy** across all six research questions.

- Values are taken from the steady-state statistics reported in Section 6
- Log scale is used due to catastrophic divergence in RQ6


In [ ]:
# --- DATA FOR FIGURE 6.1 ---

labels = [
    "RQ1\nLow-Gain",
    "RQ2\nRamanujan",
    "RQ3\nEDB-OFF",
    "RQ4\nRSI-OFF",
    "RQ5\nQuantum-ε",
    "RQ6\nUnstable"
]

values = [
    44.83,     # RQ1 Stable-LowGain
    121.38,    # RQ2 Ramanujan-d4
    62.72,     # RQ3 EDB-OFF
    63.81,     # RQ4 RSI-OFF
    42.11,     # RQ5 Quantum-epsilon
    1.6e14     # RQ6 Unstable-Fail
]

In [ ]:
# --- PLOT FIGURE 6.1 (HARDENED SAVE LOGIC) ---

# Always re-materialize directory at write time (Windows/Jupyter safe)
EXPORT_DIR = Path(EXPORT_DIR)
EXPORT_DIR.mkdir(parents=True, exist_ok=True)

if not EXPORT_DIR.exists():
    raise RuntimeError(f"Output directory missing at save time: {EXPORT_DIR}")

plt.figure(figsize=(10, 5))
plt.bar(labels, values)
plt.yscale("log")
plt.ylabel("Final Lyapunov Energy (log scale)")
plt.title("Figure 6.1 — Final Lyapunov Energy Across RQ1–RQ6")

plt.tight_layout()

plt.savefig(EXPORT_DIR / "fig_6_1_overview.svg")
plt.savefig(EXPORT_DIR / "fig_6_1_overview.jpg", dpi=FIG_DPI)
plt.show()

print("Saved: fig_6_1_overview.svg / .jpg")

## Notes on Reproducibility

- All figures are generated deterministically
- Directory creation is enforced at save-time
- Notebook is compatible with:
  - Local Jupyter
  - VS Code
  - `nbconvert --execute`
  - GitHub Actions CI

This notebook satisfies **Appendix A (formal definitions)** and **Appendix B (empirical protocol)** via explicit mapping in the paper.